In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import classification_report, multilabel_confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

import nltk
import tensorflow as tf

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
from sklearn.naive_bayes import MultinomialNB
from keras.layers import Dense, Input, LSTM, GRU, Conv1D, Dropout, Flatten, Layer, BatchNormalization
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from tensorflow import keras

from gensim.models import KeyedVectors
from gensim import models

import warnings
warnings.filterwarnings('ignore')
from imblearn.pipeline import Pipeline 
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, accuracy_score
from imblearn.metrics import classification_report_imbalanced 
from sklearn.svm import LinearSVC
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB

In [2]:
df = pd.read_csv('EI.csv')
I = 284212
E = 10827
testing = pd.concat([df.iloc[:35526], df.iloc[I:I + E]])

## Evaluation Metrics

In [10]:
# Function that evaluates the model based on the confusion matrix and accuracy score
def evaluation(X, y, model):
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
    
    model.fit(X_train,y_train)
    
    y_pred = model.predict(X_test)
    
    print('Accuracy Score:{}'.format(accuracy_score(y_test, y_pred)) )
    print(classification_report_imbalanced(y_test, y_pred))


## Pipeline Generation

In [40]:
# Logistic regression 

logistic_regression = Pipeline([('Undersample', RandomUnderSampler()), 
                                ('Logistic', LogisticRegression())])

# Support Vector Machine - L1 Regularized
svm_lasso = Pipeline([('Undersample', RandomUnderSampler()),
                    ('LinearSVC', LinearSVC(penalty='l1', dual=False))])

#Support Vector Machine - L2 Regularized
svm_ridge = Pipeline([('Undersample', RandomUnderSampler()),
                    ('LinearSVC', LinearSVC(penalty='l2', dual=False))])

# Random Forest
rf = Pipeline([('Undersample', RandomUnderSampler()),
            ('RandomForest', RandomForestClassifier(n_estimators=100, max_depth=10))])

# Naive Bayes
nb = Pipeline([('Undersample', RandomUnderSampler()),
            ('NaiveBayes', MultinomialNB())]) 

## Bag of Words

In [3]:
bow = pd.read_csv('bow.csv')

In [4]:
# define Input and Output
X_bow = bow
le = preprocessing.LabelEncoder()
y = le.fit_transform(testing['type'])

### Logistic Regression

In [41]:
evaluation(X_bow, y, logistic_regression)

Accuracy Score:0.5570799896453533
                   pre       rec       spe        f1       geo       iba       sup

          0       0.28      0.56      0.56      0.37      0.56      0.31      2754
          1       0.80      0.56      0.56      0.66      0.56      0.31      8835

avg / total       0.68      0.56      0.56      0.59      0.56      0.31     11589



### SVM - L1 Regularized

In [20]:
evaluation(X_bow, y, svm_lasso)

Accuracy Score:0.552851842264216
                   pre       rec       spe        f1       geo       iba       sup

          0       0.28      0.57      0.55      0.38      0.56      0.31      2754
          1       0.80      0.55      0.57      0.65      0.56      0.31      8835

avg / total       0.68      0.55      0.56      0.59      0.56      0.31     11589



### SVM - L2 Regularized

In [21]:
evaluation(X_bow, y, svm_ridge)

Accuracy Score:0.5424109068944689
                   pre       rec       spe        f1       geo       iba       sup

          0       0.27      0.54      0.54      0.36      0.54      0.29      2754
          1       0.79      0.54      0.54      0.64      0.54      0.29      8835

avg / total       0.67      0.54      0.54      0.58      0.54      0.29     11589



### Random Forest

In [22]:
evaluation(X_bow, y, rf)

Accuracy Score:0.6819397704719993
                   pre       rec       spe        f1       geo       iba       sup

          0       0.33      0.34      0.79      0.34      0.52      0.25      2754
          1       0.79      0.79      0.34      0.79      0.52      0.28      8835

avg / total       0.68      0.68      0.44      0.68      0.52      0.27     11589



### Multinomial Naive Bayes

In [23]:
evaluation(X_bow, y, nb)

Accuracy Score:0.5676935024592286
                   pre       rec       spe        f1       geo       iba       sup

          0       0.29      0.56      0.57      0.38      0.56      0.32      2754
          1       0.81      0.57      0.56      0.67      0.56      0.32      8835

avg / total       0.68      0.57      0.56      0.60      0.56      0.32     11589



Because we have used Undersampling method to have equal distribution of clas 0 and 1, the prediction on Class 0 (Extrovert) does not perform well

# TF-IDF

In [5]:
tfidf = pd.read_csv('tfidf.csv')

In [6]:
X_tfidf = tfidf

### Logistic Regression

In [25]:
evaluation(X_tfidf, y, logistic_regression)

Accuracy Score:0.575804642333247
                   pre       rec       spe        f1       geo       iba       sup

          0       0.29      0.53      0.59      0.37      0.56      0.31      2754
          1       0.80      0.59      0.53      0.68      0.56      0.32      8835

avg / total       0.68      0.58      0.55      0.61      0.56      0.31     11589



### SVM - L1 Regularized

In [26]:
evaluation(X_tfidf,y, svm_lasso)

Accuracy Score:0.5598412287514022
                   pre       rec       spe        f1       geo       iba       sup

          0       0.28      0.55      0.56      0.37      0.56      0.31      2754
          1       0.80      0.56      0.55      0.66      0.56      0.31      8835

avg / total       0.68      0.56      0.55      0.59      0.56      0.31     11589



### SVM - L2 Regularized

In [27]:
evaluation(X_tfidf,y, svm_ridge)

Accuracy Score:0.5489688497713349
                   pre       rec       spe        f1       geo       iba       sup

          0       0.28      0.55      0.55      0.37      0.55      0.30      2754
          1       0.80      0.55      0.55      0.65      0.55      0.30      8835

avg / total       0.67      0.55      0.55      0.58      0.55      0.30     11589



### Random Forest

In [28]:
evaluation(X_tfidf,y, rf)

Accuracy Score:0.6823712140823195
                   pre       rec       spe        f1       geo       iba       sup

          0       0.33      0.33      0.79      0.33      0.51      0.25      2754
          1       0.79      0.79      0.33      0.79      0.51      0.28      8835

avg / total       0.68      0.68      0.44      0.68      0.51      0.27     11589



### Multinomial Naive Bayes

In [29]:
evaluation(X_tfidf,y, nb)

Accuracy Score:0.559237207696954
                   pre       rec       spe        f1       geo       iba       sup

          0       0.29      0.58      0.55      0.38      0.57      0.32      2754
          1       0.81      0.55      0.58      0.66      0.57      0.32      8835

avg / total       0.68      0.56      0.57      0.59      0.57      0.32     11589



There is not significant difference between BOW and TFIDF and also the undersampling method, provide bad precision score for class 0

### Trying SMOTE - Oversampling

In [7]:
oversample = SMOTE()
X_os, y_os = oversample.fit_resample(X_tfidf, y)

### Logistic Regression

In [11]:
lr = LogisticRegression()
evaluation(X_os, y_os, lr)

Accuracy Score:0.7303383437482407
                   pre       rec       spe        f1       geo       iba       sup

          0       0.74      0.71      0.75      0.72      0.73      0.53      8864
          1       0.72      0.75      0.71      0.74      0.73      0.53      8899

avg / total       0.73      0.73      0.73      0.73      0.73      0.53     17763



### SVM - L1 Regularized

In [12]:
l1 = LinearSVC(penalty='l1',dual=False)
evaluation(X_os, y_os, l1)

Accuracy Score:0.7308450149186512
                   pre       rec       spe        f1       geo       iba       sup

          0       0.74      0.72      0.75      0.73      0.73      0.53      8864
          1       0.72      0.75      0.72      0.74      0.73      0.54      8899

avg / total       0.73      0.73      0.73      0.73      0.73      0.53     17763



### SVM - L2 Regularized

In [14]:
l2 = LinearSVC(penalty='l2',dual=False)
evaluation(X_os, y_os, l2)

Accuracy Score:0.7309013117153634
                   pre       rec       spe        f1       geo       iba       sup

          0       0.74      0.71      0.75      0.73      0.73      0.53      8864
          1       0.72      0.75      0.71      0.74      0.73      0.54      8899

avg / total       0.73      0.73      0.73      0.73      0.73      0.53     17763



### Random Forest

In [15]:
rf = RandomForestClassifier(n_estimators=100, max_depth=10)
evaluation(X_os, y_os, rf)

Accuracy Score:0.6871024038732196
                   pre       rec       spe        f1       geo       iba       sup

          0       0.67      0.73      0.65      0.70      0.69      0.47      8864
          1       0.70      0.65      0.73      0.67      0.69      0.47      8899

avg / total       0.69      0.69      0.69      0.69      0.69      0.47     17763



### Multinomial Naive Bayes

In [16]:
naive = MultinomialNB()
evaluation(X_os, y_os, naive)

Accuracy Score:0.6086246692563193
                   pre       rec       spe        f1       geo       iba       sup

          0       0.61      0.59      0.62      0.60      0.61      0.37      8864
          1       0.61      0.62      0.59      0.61      0.61      0.37      8899

avg / total       0.61      0.61      0.61      0.61      0.61      0.37     17763



As we can see, the results are better with oversampling (SMOTE) technique which significantly lncrease the precision score for class 0.
<br>
The overall accuracy score also increased with SMOTE and showing the Linear seperation method works better (Logistic, SVM) than ensemble and naive bayes model 

# Word2Vec - Pretrained

In [18]:
word2vec = pd.read_csv('word2v.csv')

In [19]:
# ### Logistic Regressionoversampling
oversample = SMOTE()
X_w2v, y_w2v = oversample.fit_resample(word2vec, y)

### Logistic Regression

In [22]:
evaluation(X_w2v, y_w2v, lr)

Accuracy Score:0.5952260316387997
                   pre       rec       spe        f1       geo       iba       sup

          0       0.59      0.62      0.57      0.60      0.59      0.36      8864
          1       0.60      0.57      0.62      0.59      0.59      0.35      8899

avg / total       0.60      0.60      0.60      0.60      0.59      0.35     17763



### SVM - L1 Regularized

In [23]:
evaluation(X_w2v, y_w2v, l1)

Accuracy Score:0.5959578899960593
                   pre       rec       spe        f1       geo       iba       sup

          0       0.59      0.62      0.57      0.61      0.60      0.36      8864
          1       0.60      0.57      0.62      0.59      0.60      0.35      8899

avg / total       0.60      0.60      0.60      0.60      0.60      0.35     17763



### SVM - L2 Regularized

In [24]:
evaluation(X_w2v, y_w2v, l2)

Accuracy Score:0.5964645611664696
                   pre       rec       spe        f1       geo       iba       sup

          0       0.59      0.62      0.57      0.61      0.60      0.36      8864
          1       0.60      0.57      0.62      0.59      0.60      0.35      8899

avg / total       0.60      0.60      0.60      0.60      0.60      0.36     17763



### Random Forest

In [25]:
evaluation(X_w2v, y_w2v, rf)

Accuracy Score:0.69306986432472
                   pre       rec       spe        f1       geo       iba       sup

          0       0.69      0.70      0.68      0.70      0.69      0.48      8864
          1       0.70      0.68      0.70      0.69      0.69      0.48      8899

avg / total       0.69      0.69      0.69      0.69      0.69      0.48     17763



Pretraiend Word2vec model performance was not great overall and was in fact lower than tfidf with SMOTE.

# Hybrid - Word2Vec * TFIDF

In [28]:
hybrid = pd.read_csv('hybrid.csv')

In [29]:
oversample = SMOTE()
X_hybrid, y_hybrid = oversample.fit_resample(hybrid, y)

In [30]:
evaluation(X_hybrid, y_hybrid, lr)

Accuracy Score:0.71119743286607
                   pre       rec       spe        f1       geo       iba       sup

          0       0.69      0.76      0.66      0.72      0.71      0.51      8864
          1       0.74      0.66      0.76      0.70      0.71      0.50      8899

avg / total       0.71      0.71      0.71      0.71      0.71      0.50     17763



In [31]:
evaluation(X_hybrid, y_hybrid, l1)

Accuracy Score:0.708044812250183
                   pre       rec       spe        f1       geo       iba       sup

          0       0.68      0.77      0.65      0.72      0.71      0.50      8864
          1       0.74      0.65      0.77      0.69      0.71      0.49      8899

avg / total       0.71      0.71      0.71      0.71      0.71      0.50     17763



In [32]:
evaluation(X_hybrid, y_hybrid, l2)

Accuracy Score:0.7076507346731971
                   pre       rec       spe        f1       geo       iba       sup

          0       0.68      0.77      0.65      0.72      0.71      0.50      8864
          1       0.74      0.65      0.77      0.69      0.71      0.49      8899

avg / total       0.71      0.71      0.71      0.71      0.71      0.50     17763



In [33]:
evaluation(X_hybrid, y_hybrid, rf)

Accuracy Score:0.7548837471147891
                   pre       rec       spe        f1       geo       iba       sup

          0       0.76      0.75      0.76      0.75      0.75      0.57      8864
          1       0.75      0.76      0.75      0.76      0.75      0.57      8899

avg / total       0.75      0.75      0.75      0.75      0.75      0.57     17763



Results show that hybrid mechansim works well with this data and Random Forest had the highest prediction score. We can try to implement Boosting techniques to see if other tree models works better or do Hyperparameter tunning to increase the performance